In [39]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
from numpy import zeros_like
from IPython.display import display
def display_matrix(m):
    display(sympy.Matrix(m))
import sympy
import pandas as pd
import scipy.stats as ss

sympy.init_printing()

In [40]:
basis = 'leguerre'

# parameters

mu = 0.06
r = 0.06
T = 1
I = 250
paths = 10000
S0 = 40
S0_list = range(37, 43, 1)
m = 0
K = 40

GBM_sigma = 0.04
GBM_mu = r

LN_lam = 1
LN_sigma = 0.02
LN_mu = r
LN_v = 0.02

JR_lam = 0.01
JR_sigma = 0.03
JR_mu = r + JR_lam

LNparams = (LN_lam, LN_sigma, LN_mu, LN_v, m)
JRparams = (JR_lam, JR_sigma, JR_mu, 0, m)
GBMparams = (GBM_mu, GBM_sigma)

In [41]:
def merton_jump_paths_sample(S0, paths, I, T, mu, sigma, lam, m, v):
    np.random.seed(42)
    dt = T / I
    matrix = np.zeros((paths, I))
    all_normals = []
    all_lognormals = []
    
    for k in range(paths):
        X = np.zeros(I)
        S = np.zeros(I)
        S[0] = S0
        X[0] = np.log(S0)

        for i in range(1, I):
            Z = np.random.standard_normal()
            all_normals.append(Z)
            
            N = np.random.poisson(lam * dt)
            if N > 0:
                Y = np.exp(np.random.normal(m, v, N))
                logY = np.log(Y)
                M = np.sum(np.log(Y))
                all_lognormals.extend(logY)
            else:
                M = 0

            X[i] = X[i-1] + (mu - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z + M
            S[i] = np.exp(X[i])
        matrix[k] = S

    mean_normal = np.mean(all_normals)
    std_normal = np.std(all_normals)

    if all_lognormals:
        mean_lognormal = np.mean(all_lognormals)
        std_lognormal = np.std(all_lognormals)
    else:
        mean_lognormal = 0
        std_lognormal = 0

    return mean_normal, mean_lognormal, std_normal, std_lognormal


In [42]:
mean_normal_LN, mean_lognormal_LN, std_normal_LN, std_lognormal_LN = merton_jump_paths_sample(S0, paths, I, T, LN_mu, LN_sigma, LN_lam, m, LN_v)

In [43]:
merton_jump_paths_sample(S0, paths, I, T, LN_mu, LN_sigma, LN_lam, m, LN_v)

In [44]:
def merton_jump_paths_mm(S0, paths, I, T, mu, sigma, lam, m, v, mean_normal_LN, mean_lognormal_LN, std_normal_LN, std_lognormal_LN):
    np.random.seed(42)
    S0, paths, I, T = S0, paths, I, T
    matrix = np.zeros((paths, I))
    for k in range(paths):
        X = np.zeros(I)
        S = np.zeros(I)
        S[0] = S0
        X[0] = np.log(S0)
        dt = T / I
        for i in range(1,I):
            Z = np.random.standard_normal()
            N = np.random.poisson(lam * dt)
            Y = np.exp(np.random.normal(m,v,N))
            logy = np.log(Y)

            if N == 0:
                M = 0
            else:
                M = np.sum(((logy - mean_lognormal_LN)/std_lognormal_LN)*v+m)
            X[i] = X[i-1] + (mu - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * ((Z-mean_normal_LN)/std_normal_LN) + M
            S[i] = np.exp(X[i])
        matrix[k] = S
    return matrix

In [45]:
def merton_jump_paths(S0, paths, lam, sigma, mu, v, m, T, I):
    np.random.seed(42)
    matrix = np.zeros((paths, I))
    for k in range(paths):
        X = np.zeros(I)
        S = np.zeros(I)
        
        S[0] = S0
        X[0] = np.log(S0)
        
        dt = T / I
    
        for i in range(1,I):
            Z = np.random.standard_normal()
            N = np.random.poisson(lam * dt)
            Y = np.exp(np.random.normal(m,v,N))
            #Y = np.random.lognormal(m,np.sqrt(v),N)
    
            if N == 0:
                M = 0
            else:
                M = np.sum(np.log(Y))
                
            X[i] = X[i-1] + (mu - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z + M
            S[i] = np.exp(X[i])
        matrix[k] = S
    return matrix

In [46]:
def LSM(K, S, I, df, basis, deg):
    paths = len(S)
    np.random.seed(42)
    H = np.maximum(K - S, 0)  # intrinsic values for put option
    V = np.zeros_like(H)  # value matrix
    V[:, -1] = H[:, -1]  # set value at maturity equal to intrinsic value

    # Valuation by LS Method
    for t in range(I - 2, 0, -1):
        good_paths = H[:, t] > 0  # paths where the intrinsic value is positive

        if np.sum(good_paths) > 0:
            if basis == 'poly':
                rg = np.polyfit(S[good_paths, t], V[good_paths, t + 1] * df, deg)
                C = np.polyval(rg, S[good_paths, t])
            elif basis == 'legendre':
                rg = np.polynomial.legendre.legfit(S[good_paths, t], V[good_paths, t + 1] * df, deg)
                C = np.polynomial.legendre.legval(S[good_paths, t], rg)
            elif basis =='laguerre':
                rg = np.polynomial.laguerre.lagfit(S[good_paths, t], V[good_paths, t + 1] * df, deg)
                C = np.polynomial.laguerre.lagval(S[good_paths, t], rg)
            else:  # 'hermite'
                rg = np.polynomial.hermite.hermfit(S[good_paths, t], V[good_paths, t + 1] * df, deg)
                C = np.polynomial.hermite.hermval(S[good_paths, t], rg)

            exercise = np.zeros(len(good_paths), dtype=bool)
            exercise[good_paths] = H[good_paths, t] > C
        else:
            # If all intrinsic values are zero, mark all as non-exercise
            exercise = np.zeros(len(good_paths), dtype=bool)

        V[exercise, t] = H[exercise, t]
        V[exercise, t + 1 :] = 0
        discount_path = ~exercise
        V[discount_path, t] = V[discount_path, t + 1] * df

    V0 = np.mean(V[:, 1]) * df  # discounted expectation of V[t=1]
    V0_array = V[:, 1] * df
    SE = np.std(V[:, 1] * df) / np.sqrt(paths)
    variance = np.var(V[:, 1] * df)
    return V0, V0_array, SE, variance

In [47]:
paths_mm = merton_jump_paths_mm(S0, paths, I, T, LN_mu, LN_sigma, LN_lam, m, LN_v, mean_normal_LN, mean_lognormal_LN, std_normal_LN, std_lognormal_LN)
paths = merton_jump_paths(S0, paths, LN_lam, LN_sigma, LN_mu, LN_v, m, T, I)

mm_V0, mm_V0_array, mm_SE, mm_variance = LSM(K, paths_mm, I, np.exp(-r * T), basis, 3)
V0, V0_array, SE, variance = LSM(K, paths, I, np.exp(-r * T), basis, 3)

In [48]:
print('MM V0: ', mm_V0.round(5))
print('MM SE: ', mm_SE.round(5))
print('MM variance: ', mm_variance.round(5))

print('V0: ', V0.round(5))
print('SE: ', SE.round(5))
print('variance: ', variance.round(5))

MM V0:  0.03885
MM SE:  0.0008
MM variance:  0.00643
V0:  0.0389
SE:  0.00081
variance:  0.00652
